In [ ]:
#Vikas Bansal
#10 April 2020
#This script is used to read the metric file from Cell Ranger and plot the heatmap (nUMI, nCells etc) for each sample

In [1]:
set.seed(7860)
.libPaths( c( "~/Rlib/", .libPaths()) )
library(readxl)
library(stringi)
library(ComplexHeatmap)

Loading required package: grid
ComplexHeatmap version 2.2.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.



In [2]:

#Path to metric files from cell ranger output 
Gene_results_file_names <- list.files(path="/home/vikas/QC_FOUNDIN/Metrics_Summary/",recursive = T, full=T)



In [3]:
#Read metric files from cell ranger output
test_count <- read.csv(Gene_results_file_names[1], stringsAsFactors=F, header=T)
rownames(test_count)[1] <- strsplit(unlist(lapply(strsplit(Gene_results_file_names[1],"/"),tail,3)),"_count")[[1]][1]
ALL_WITH_COUNT <- test_count
for (i in 2:length(Gene_results_file_names)){
  #cat("loop", i, "\n")
  
  
  test_count <- read.csv(Gene_results_file_names[i], stringsAsFactors=F, header=T)
  
  rownames(test_count)[1] <- strsplit(unlist(lapply(strsplit(Gene_results_file_names[i],"/"),tail,3)),"_count")[[1]][1]
  ALL_WITH_COUNT <- rbind(ALL_WITH_COUNT,test_count)
  
}

#Put % sign in column names to keep track which columns contain percentages
colnames(ALL_WITH_COUNT)[(grep("%",ALL_WITH_COUNT))] <- paste0(colnames(ALL_WITH_COUNT)[(grep("%",ALL_WITH_COUNT))],"%")

#Remove % and , from the whole data frame to convert them into numeric
ALL_WITH_COUNT[] <- lapply(ALL_WITH_COUNT, gsub, pattern = "%", replacement = "", fixed = TRUE)
ALL_WITH_COUNT[] <- lapply(ALL_WITH_COUNT, gsub, pattern = ",", replacement = "", fixed = TRUE)

In [4]:
#convert columns into numeric and save to new dataframe
ALL_WITH_COUNTv2 <- sapply( ALL_WITH_COUNT, as.numeric )
rownames(ALL_WITH_COUNTv2) <- rownames(ALL_WITH_COUNT)

In [5]:
#Read metaInfo for samples to keep track of batches
Petertable <- read_excel("Samples_names_codes_final_v.Cornelis.xlsx", sheet = 1)
Petertablev2 <- (as.data.frame(Petertable))
Petertablev3 <- Petertablev2[grep("CDI",Petertablev2[,"Barcode_DZNE"]),]


Petertablev3$Barcode_last4 <- (stri_sub(Petertablev3$Barcode_DZNE,-4,-1))

Petertablev3$SampleID <- (paste0("SCRN_PPMI",Petertablev3$PPMI_ID, "_", Petertablev3$Barcode_last4,"_da65"))


In [6]:
#Put annotation for multiple control samples (SCRN_PPMI3966_2813_da65). Also change the batch column
Petertablev4 <- Petertablev3
Petertablev4[c(nrow(Petertablev3)+1,nrow(Petertablev3)+2,nrow(Petertablev3)+3),] <- Petertablev3[grep("SCRN_PPMI3966_2813_da65",Petertablev3$SampleID),]
Petertablev4[c(nrow(Petertablev3)+1,nrow(Petertablev3)+2,nrow(Petertablev3)+3),"SampleID"] <- rownames(ALL_WITH_COUNTv2)[which(is.na(match(rownames(ALL_WITH_COUNTv2),Petertablev3$SampleID)))]
Petertablev4[grep("SCRN_PPMI3966B3_2813_da65",Petertablev4$SampleID),"BATCH"] <- 3
Petertablev4[grep("SCRN_PPMI3966E6_2813_da65",Petertablev4$SampleID),"BATCH"] <- 5
Petertablev4[grep("SCRN_PPMI3966E8_2813_da65",Petertablev4$SampleID),"BATCH"] <- 5


In [7]:
#Creating metadata in same order to plot it over heatmap
metaDataHeatmap <- Petertablev4[match(rownames(ALL_WITH_COUNTv2),Petertablev4$SampleID),]




In [8]:
#Scaling the dataframe column wise, meaning for each variable (not sample wise)
ALL_WITH_COUNTv2_scaled = t(scale(ALL_WITH_COUNTv2))

In [9]:
#Creating upper annotation for heatmap
ha = HeatmapAnnotation(Batch = as.factor(metaDataHeatmap$BATCH), 
    pheno = as.factor(metaDataHeatmap$pheno),
    RECRUITMENT_CAT = metaDataHeatmap$RECRUITMENT_CAT,
    genetic_sex = as.factor(metaDataHeatmap$genetic_sex),
    mutation = metaDataHeatmap$mutation, col=list(Batch=c("1"="Red","2"="Blue","3"="Grey","4"="Purple","5"="Black"),
                                                 pheno=c("-9"='#0B5390FF',"1"='#708605FF',"2"='#089B17FF'),
                                                 RECRUITMENT_CAT=c("GENPD"='#F24C5CFF',"GENUN"='#55D5C0FF',"HC"='#E81BF8FF',"PD"='#A11FCEFF',"PRODROMA"='#51ED99FF'),
                                                 genetic_sex=c("1"='#4814C9FF',"2"='#EA6774FF'),
                                                 mutation=c("GBA_N409S"='#F166E7FF',"LRRK2_Arg1441Gly"='#08276FFF',"LRRK2_G2019S"='#8E7719FF',"na"='grey',"SNCA_A53T"='#515152FF'))

)


In [10]:
pdf("HeatmapNumberStats.pdf", width=30, height=10)

Heatmap(ALL_WITH_COUNTv2_scaled[c(1:3,19:20),], name = "Z-score", km = 1, top_annotation = ha, 
     show_row_names = TRUE, 
    show_column_names = TRUE, show_row_dend = FALSE, cluster_rows = FALSE, cluster_row_slices = FALSE)
dev.off()

png 
  2

In [11]:
pdf("HeatmapPercentStats.pdf", width=30, height=10)

Heatmap(ALL_WITH_COUNTv2_scaled[-c(1:3,19:20),], name = "Z-score", km = 1, top_annotation = ha, 
     show_row_names = TRUE, 
    show_column_names = TRUE, show_row_dend = FALSE, cluster_rows = FALSE, cluster_row_slices = FALSE)
dev.off()

png 
  2

In [12]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 16.04.5 LTS

Matrix products: default
BLAS/LAPACK: /opt/anaconda3/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] ComplexHeatmap_2.2.0 stringi_1.4.3        readxl_1.3.1        

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1          cluster_2.0.8       uuid_0.1-2         
 [4] colorspace_1.4-1    clue_0.3-57         rjson_0.2.20       
 [7] rlang_0.3.4         tools_3.6.1         parallel_3.6.1     
[10] c